#Use isto se for a primeira vez usando o datalake da basedosdados.org: <br>
   pip install basedosdados
<br><br>
#Use para baixar a base como microdados_violencia.csv: <br>
   import basedosdados as bd <br>
   df = bd.read_table(dataset_id='br_ms_sinan', table_id='microdados_violencia', billing_project_id="violenciasinan", use_bqstorage_api = True) <br>
   df.to_csv(path_or_buf = 'microdados_violencia.csv')

In [16]:
# importar bibliotecas usadas
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing

In [17]:
print(tf.__version__)
try:
    physical_devices = tf.config.list_physical_devices('GPU') 
    #tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print(physical_devices)
except:
    print("No GPU")

2.9.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [18]:
# carregar dados em um dataframe pandas
# para usar o arquivo .csv baixe da pasta do sharepoint ou com os comandos disponíveis no topo do notebook e insira o caminho do arquivo no comando abaixo
df = pd.read_csv("microdados_violencia.csv")
# excluir colunas inadequadas para a análise
df = df.drop(['data_encerramento', 'data_notificacao', 'id_categoria_cid10', 'id_subcategoria_cid10', 'data_ocorrencia', 'hora_ocorrencia', 'id_municipio_notificacao', 'id_municipio_6_notificacao', 'id_unidade_notificacao',
'id_regional_saude_notificacao', 'id_municipio_ocorrencia', 'id_municipio_6_ocorrencia', 'id_municipio_residencia', 'id_municipio_6_residencia', 'id_regional_saude_residencia',
'houve_qual_outra_violencia_sexual', 'meio_qual_outro', 'ocorreu_qual_outra', 'outro_local_ocorrencia', 'quais_outras_deficiencias_paciente', 'autor_relacao_outros'], axis=1)

c:\Users\rozan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
# resumo dos dados
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df)

,Unnamed: 0,ano,tipo_notificacao,id_uf_notificacao,id_uf_ocorrencia,local_ocorrencia,outras_vezes_ocorrencia,id_uf_residencia,idade_paciente,sexo_paciente,gestante_paciente,raca_paciente,escolaridade_paciente,ocupacao_paciente,estado_civil_paciente,orientacao_sexual_paciente,identidade_genero_paciente,motivacao_violencia,violencia_relacionada_trabalho,emitiu_cat,deficiencia_transtorno_paciente,deficiencia_fisica_paciente,deficiencia_mental_paciente,deficiencia_visual_paciente,deficiencia_auditiva_paciente,transtorno_mental_paciente,transtorno_comportamental_paciente,outras_deficiencias_paciente,lesao_autoprovocada,ocorreu_violencia_fisica,ocorreu_violencia_psicologica,ocorreu_tortura,ocorreu_violencia_sexual,ocorreu_trafico_ser_humano,ocorreu_violencia_financeira,ocorreu_negligencia_abandono,ocorreu_trabalho_infantil,ocorreu_intervencao_legal,ocorreu_outra_violencia,meio_forca,meio_enforcamento,meio_objeto_contundente,meio_objeto_perfurante,meio_objeto_quente,meio_envenenamento,meio_arma_fogo,meio_ameaca,meio_outros,houve_assedio,houve_estupro,houve_pornografia_infantil,houve_exploracao_sexual,houve_outra_violencia_sexual,profilaxia_dst,profilaxia_hiv,profilaxia_hepatite_b,coleta_sangue,coleta_semen,coleta_secrecao_vaginal,profilaxia_contraceptivo,aborto,numero_envolvidos_violencia,autor_pai,autor_mae,autor_padrasto,autor_madrasta,autor_conjugue,autor_ex_conjugue,autor_namorado_a,autor_ex_namorado_a,autor_filho_a,autor_desconhecido,autor_irmao,autor_conhecido,autor_cuidador,autor_patrao_chefe,autor_institucional,autor_policial,autor_propria_pessoa,autor_outros,autor_sexo,autor_usou_alcool,encaminhamento_saude,encaminhamento_assistencia_social,encaminhamento_educacao,encaminhamento_atendimento_mulher,encaminhamento_conselho_tutelar,encaminhamento_conselho_idoso,encaminhamento_delegacia_idoso,encaminhamento_direitos_humanos,encaminhamento_mpu,encaminhamento_delegacia_crianca,encaminhamento_delegacia_mulher,encaminhamento_delegacia,encaminhamento_justica_infancia_juventude,encaminhamento_defensoria_publica
0,0,2009,2,12,12.0,6.0,0.0,12,26.0,1.0,6.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,0.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
1,1,2009,2,27,27.0,NaN,NaN,27,27.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299140,2299140,2016,2,33,33.0,1.0,1.0,33,11.0,0.0,5.0,4.0,3.0,999991.0,1.0,1.0,8.0,88.0,0.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2299141,2299141,2016,2,33,33.0,1.0,0.0,33,17.0,1.0,6.0,2.0,3.0,NaN,NaN,1.0,8.0,NaN,0.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
inputs

{'Unnamed: 0': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Unnamed: 0')>,
 'ano': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'ano')>,
 'id_subcategoria_cid10': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'id_subcategoria_cid10')>,
 'tipo_notificacao': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'tipo_notificacao')>,
 'id_uf_notificacao': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'id_uf_notificacao')>,
 'data_ocorrencia': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'data_ocorrencia')>,
 'id_uf_ocorrencia': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'id_uf_ocorrencia')>,
 'hora_ocorrencia': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'hora_ocorrencia')>,
 'local_ocorrencia': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'local_ocorrencia')>,
 'id_uf_residencia': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'id_u

In [21]:
# separando labels e features
features = df.copy()
labels = features.pop('outras_vezes_ocorrencia')

In [22]:
inputs = {}

for name, column in features.items():
  dtype = column.dtype
  if dtype == object:
    features.drop(name, axis=1)
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

In [23]:
# modelo de regressão
# SGD como otimizador
modelo = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

modelo.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.SGD())

In [24]:
modelo.fit(features, labels, epochs=10)

Epoch 1/10
71849/71849 [==============================] - 158s 2ms/step - loss: nan
Epoch 2/10
71849/71849 [==============================] - 158s 2ms/step - loss: nan
Epoch 3/10
71849/71849 [==============================] - 155s 2ms/step - loss: nan
Epoch 4/10
71849/71849 [==============================] - 151s 2ms/step - loss: nan
Epoch 5/10
71849/71849 [==============================] - 154s 2ms/step - loss: nan
Epoch 6/10
71849/71849 [==============================] - 146s 2ms/step - loss: nan
Epoch 7/10
71849/71849 [==============================] - 151s 2ms/step - loss: nan
Epoch 8/10
71849/71849 [==============================] - 154s 2ms/step - loss: nan
Epoch 9/10
71849/71849 [==============================] - 149s 2ms/step - loss: nan
Epoch 10/10
71849/71849 [==============================] - 148s 2ms/step - loss: nan


In [27]:
modelo.save('./pesos/')

INFO:tensorflow:Assets written to: ./pesos/assets
